In [ ]:
!pip install convokit
!pip install transformers
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216708 sha256=fcc94293532fbd53be415ce8829abd9cbb02585e9bb8bfec6abf066ee5388d73
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1c33e0d5525b231
  

In [ ]:
from convokit import Corpus, download
se_corpus = Corpus(filename=download("stack-exchange-politeness-corpus"))


No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
default_backend: mem


In [ ]:
wiki_corpus = Corpus(filename=download("wikipedia-politeness-corpus"))


In [ ]:
import pandas as pd

def extract_politeness_data(corpus):
    data = []
    for utt in corpus.iter_utterances():
        text = utt.text
        score = utt.meta['Normalized Score']
        data.append({'text': text, 'score': score})
    return pd.DataFrame(data)


In [ ]:
se_df = extract_politeness_data(se_corpus)
wiki_df = extract_politeness_data(wiki_corpus)


In [ ]:
def label_data(df):
    q1 = df['score'].quantile(0.25)
    q3 = df['score'].quantile(0.75)

    def label_score(score):
        if score <= q1:
            return 'impolite'
        elif score >= q3:
            return 'polite'
        else:
            return 'neutral'

    df['label'] = df['score'].apply(label_score)
    return df

se_df = label_data(se_df)
wiki_df = label_data(wiki_df)


In [ ]:
def filter_data(df):
    return df[df['label'] != 'neutral'].reset_index(drop=True)

se_df_filtered = filter_data(se_df)
wiki_df_filtered = filter_data(wiki_df)


In [ ]:
from sklearn.model_selection import train_test_split

def split_data(df):
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
    return train_df, test_df

se_train_df, se_test_df = split_data(se_df_filtered)
wiki_train_df, wiki_test_df = split_data(wiki_df_filtered)

In [ ]:
from convokit import PolitenessStrategies

ps = PolitenessStrategies()

def extract_features(corpus, df):
    corpus = ps.transform(corpus)
    features = []
    labels = []
    for utt in corpus.iter_utterances():
        if utt.text in df['text'].values:
            features.append(utt.meta['politeness_strategies'])
            labels.append(df[df['text'] == utt.text]['label'].values[0])
    feature_df = pd.DataFrame(features)
    feature_df['label'] = labels
    return feature_df.fillna(0)


In [ ]:
def update_corpus(corpus, df):
    texts = set(df['text'].values)
    utterances = [utt for utt in corpus.iter_utterances() if utt.text in texts]
    corpus_filtered = Corpus(utterances=utterances)
    return corpus_filtered


In [ ]:
se_train_corpus = update_corpus(se_corpus, se_train_df)
se_test_corpus = update_corpus(se_corpus, se_test_df)

wiki_train_corpus = update_corpus(wiki_corpus, wiki_train_df)
wiki_test_corpus = update_corpus(wiki_corpus, wiki_test_df)

In [ ]:
se_train_features = extract_features(se_train_corpus, se_train_df)
se_test_features = extract_features(se_test_corpus, se_test_df)

wiki_train_features = extract_features(wiki_train_corpus, wiki_train_df)
wiki_test_features = extract_features(wiki_test_corpus, wiki_test_df)


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

def train_svm(train_features):
    X_train = train_features.drop('label', axis=1)
    y_train = train_features['label']
    clf = LinearSVC()
    clf.fit(X_train, y_train)
    return clf


In [ ]:
se_clf = train_svm(se_train_features)
wiki_clf = train_svm(wiki_train_features)

In [ ]:
def evaluate_model(clf, test_features):
    X_test = test_features.drop('label', axis=1)
    y_test = test_features['label']
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

se_ling_accuracy = evaluate_model(se_clf, se_test_features)
wiki_ling_accuracy = evaluate_model(wiki_clf, wiki_test_features)

print(f"Stack Exchange Ling Model Accuracy: {se_ling_accuracy*100:.2f}%")
print(f"Wikipedia Ling Model Accuracy: {wiki_ling_accuracy*100:.2f}%")


Stack Exchange Ling Model Accuracy: 57.74%
Wikipedia Ling Model Accuracy: 73.58%


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import Dataset
import torch
class PolitenessDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
        self.labels = labels.apply(lambda x: 1 if x == 'polite' else 0).tolist()

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).long()
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
se_train_dataset = PolitenessDataset(se_train_df['text'], se_train_df['label'])
se_test_dataset = PolitenessDataset(se_test_df['text'], se_test_df['label'])

wiki_train_dataset = PolitenessDataset(wiki_train_df['text'], wiki_train_df['label'])
wiki_test_dataset = PolitenessDataset(wiki_test_df['text'], wiki_test_df['label'])


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

def train_bert(train_dataset, eval_dataset):
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        logging_dir='./logs',
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        greater_is_better=True,
        save_total_limit=1,
        # The fix: Set save_strategy to "epoch" to match evaluation_strategy
        save_strategy="epoch",
    )

    def compute_metrics(p):
        preds = p.predictions.argmax(-1)
        labels = p.label_ids
        acc = (preds == labels).mean()
        return {'accuracy': acc}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer

In [ ]:
!pip install torch

In [ ]:
print("Training BERT on Stack Exchange data:")
se_trainer = train_bert(se_train_dataset, se_test_dataset)


Training BERT on Stack Exchange data:


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.625000,0.586341,0.689864
2,0.423200,0.747455,0.629349
3,0.217800,0.873713,0.688351


In [ ]:
print("Training BERT on Wikipedia data:")
wiki_trainer = train_bert(wiki_train_dataset, wiki_test_dataset)


Training BERT on Wikipedia data:


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.347200,0.332483,0.871560
2,0.101100,0.464338,0.876147


Epoch,Training Loss,Validation Loss,Accuracy
1,0.347200,0.332483,0.871560
2,0.101100,0.464338,0.876147


In [ ]:
def evaluate_bert(trainer, test_dataset):
    metrics = trainer.evaluate(eval_dataset=test_dataset)
    accuracy = metrics['eval_accuracy']
    print(f"Test Accuracy: {accuracy*100:.2f}%")
    return accuracy

se_bert_accuracy = evaluate_bert(se_trainer, se_test_dataset)
wiki_bert_accuracy = evaluate_bert(wiki_trainer, wiki_test_dataset)


Test Accuracy: 68.99%


Test Accuracy: 89.22%


In [ ]:
import os
import openai


openai.api_key = 'sk-zdV6D5FqM-TU2j_PPQsJzA'

openai.api_base = 'https://cmu.litellm.ai'

# (Optional) Set the API type if required by the proxy
openai.api_type = 'open_ai'  # Adjust if necessary




In [ ]:
def get_chatgpt_prediction(text):
    prompt = f"Do a binary classification on the politeness for the given text. If it's polite then output 1. Otherwise output 0 for impolite.\n\n{text}"

    response = openai.ChatCompletion.create(
        model="gpt-4o",  # Use a model supported by the proxy
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=1,
        temperature=0,
    )

    prediction = response['choices'][0]['message']['content'].strip()
    return prediction


In [ ]:
import time

def get_predictions(df):
    predictions = []
    for idx, row in df.iterrows():
        text = row['text']
        prediction = get_chatgpt_prediction(text)
        predictions.append(prediction)
        time.sleep(1)  # Adjust based on rate limits
    return predictions


In [ ]:
se_chatgpt_predictions_raw = get_predictions(se_test_df)
wiki_chatgpt_predictions_raw = get_predictions(wiki_test_df)


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
def process_predictions(predictions):
    processed = []
    for pred in predictions:
        if '1' in pred:
            processed.append('polite')
        elif '0' in pred:
            processed.append('impolite')
        else:
            processed.append('unknown')
    return processed

se_chatgpt_predictions = process_predictions(se_chatgpt_predictions_raw)
wiki_chatgpt_predictions = process_predictions(wiki_chatgpt_predictions_raw)

def evaluate_chatgpt(df, predictions):
    df = df.copy()
    df['prediction'] = predictions
    df = df[df['prediction'] != 'unknown']
    accuracy = (df['label'] == df['prediction']).mean()
    return accuracy

se_chatgpt_accuracy = evaluate_chatgpt(se_test_df, se_chatgpt_predictions)
wiki_chatgpt_accuracy = evaluate_chatgpt(wiki_test_df, wiki_chatgpt_predictions)

print(f"Stack Exchange ChatGPT Accuracy: {se_chatgpt_accuracy*100:.2f}%")
print(f"Wikipedia ChatGPT Accuracy: {wiki_chatgpt_accuracy*100:.2f}%")
